In [162]:
%matplotlib notebook
import numpy as np
import matplotlib
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotting_resources as res
import matplotlib.gridspec as gs
plt.rc('text', usetex=True)
plt.ioff()

rauto_cut = 22.9
frauto_cut = 10**((rauto_cut-23.9)/(-2.5))

In [156]:
member_args = {'ms': 8, 'markeredgewidth':0.5, 'markeredgecolor': 'k'}
field_args = {'ms':2}

# Load the catalogs and merge U-B info

In [157]:
slitcat = Table().read('../catalogs/personal_catalogs/slits_phot_zs_cutonslitdist_rcl.csv', format='csv')
ldpcat = slitcat[np.where((slitcat['Rauto'] < rauto_cut) & 
                          (slitcat['Q'] >= 4) & 
                          (slitcat['slit_distance'] < 1.0))]
cltable = res.load_clusters()
rfmags = Table().read('../catalogs/eazy_colors/ediscs_zldp_zfors_EAzY_outputs.csv', format='csv')
rfmags.rename_column('id', 'ids')

fulltable = join(ldpcat, rfmags['ids','MAG_AB_rfB', 'MAG_AB_rfU'], keys='ids', join_type='left')
fulltable['UB_color'] = fulltable['MAG_AB_rfU'] - fulltable['MAG_AB_rfB']

# Build the combined field sample

In [158]:
# Chose clusters to use
cltable.sort('z')
fieldcls = cltable[0:1]

for cl in cltable:
    if cl['z'] > fieldcls['z'][-1] + 0.04:
        fieldcls.add_row(cl)

In [159]:
# Build the catalogs
fieldsample = Table()
for cl in fieldcls:
    subtable = fulltable[np.where((fulltable['field'] == cl['tablename']) &
                                  (abs(fulltable['zLDP']-cl['z']) > 0.02))]
    fieldsample = vstack([fieldsample, subtable])

# Make the plots

In [160]:
def turn_off_tick_labels(ax_obj, ax):
    if ax == 'x':
        ax_obj.set_xticklabels([])
    elif ax == 'y':
        ax_obj.set_yticklabels([])

In [163]:
fig = plt.figure(figsize=(8.5, 11))
outer = gs.GridSpec(7, 3, wspace=0.05, hspace=0.1)

fig.text(0.5, 0.06, r'$M_B$', ha='center')
fig.text(0.06, 0.5, r'$U-B$', va='center', rotation='vertical')

for i in range(21):
    inner = gs.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[i], wspace=0)
    a1 = plt.subplot(inner[0]) 
    a2 = plt.subplot(inner[1])
        
    cl = cltable[i]
    subcat = fulltable[np.where((fulltable['field'] == cl['tablename']) &
                                (abs(fulltable['zLDP']-cl['z']) < 0.02))]
    subfield = fieldsample[np.where(abs(fieldsample['zLDP']-cl['z']) < 0.02)]
    red, blue = res.add_red_blue(subcat, cl)
    redf, bluef = res.add_red_blue(subfield, cl)
    
    # Plot left panel
    a1.plot(red['MAG_AB_rfB'], red['UB_color'], '.r', **member_args)
    a1.plot(blue['MAG_AB_rfB'], blue['UB_color'], '.b', **member_args)
    a1.axhline(cl['cmr'], color='k', lw=0.5, ls='--')
    
    # Format left panel
    a1.set_ylim(0, 2.3)
    a1.set_xlim(-23, -18)
    a1.tick_params(direction="in")
    a1.set_yticks([0.0, 0.5, 1.0, 1.5, 2.0])
    a1.set_xticks([-20, -22])
    a1.text(-22.5, 2.0, r'{}'.format(cl['papername']), usetex=True)
    
    # Plot the right panel
    a2.plot(redf['MAG_AB_rfB'], redf['UB_color'], '.r', **field_args)
    a2.plot(bluef['MAG_AB_rfB'], bluef['UB_color'], '.b', **field_args)
    a2.axhline(cl['cmr'], color='k', lw=0.5, ls='--')
    
    # Format the right panel
    a2.set_ylim(0, 2.3)
    a2.set_xlim(-23, -18)
    a2.set_yticklabels([])
    a2.tick_params(direction="in")
    a2.set_xticks([-20, -22])
    a2.text(-22.5, 2.0, r'$z={}$'.format(cl['z']), usetex=True)
    
    # Turn off the appropriate tick labels
    if i not in [18, 19, 20]:
        turn_off_tick_labels(a1, 'x')
        turn_off_tick_labels(a2, 'x')
    if i not in [0, 3, 6, 9, 12, 15, 18]:
        turn_off_tick_labels(a1, 'y')
        turn_off_tick_labels(a2, 'y')

In [164]:
plt.savefig('plots/big_cmr.eps', dpi=800)